# This dashboard provides the top 10 state occupations using the BLS 2023 Occupational Employment and Wage Statistics Dataset
## Simply enter your state of choice and get acess to the dashboard

In [51]:
import dash
from dash import dcc, html, Input, Output
import psycopg2
import pandas as pd
import plotly.express as px

# Database connection details
db_connection = {
    "host": "localhost",
    "dbname": "BLS_HCC_Project",
    "port": "5432",
    "user": "postgres",
    "password": "5432"
}

# Function to fetch data for top occupations with mean annual wage
def get_top_occupations(state):
    # Properly insert the state input into the query
    query = '''
    SELECT "occupational_title", MAX("mean_annual_wage") AS "mean_annual_wage"
    FROM "2023_State_Occupational_Employment_and_Wage_Statistics"
    WHERE "area_title" = %s AND "mean_annual_wage" IS NOT NULL
    GROUP BY "occupational_title"
    ORDER BY "mean_annual_wage" DESC
    LIMIT 10; '''

    try:
        # Connect to the database
        conn = psycopg2.connect(**db_connection)
        
        # Fetch data into a DataFrame
        df = pd.read_sql_query(query, conn, params=(state,))
        conn.close()

        # Ensure the data is numeric and handle any conversion errors
        df['mean_annual_wage'] = pd.to_numeric(df['mean_annual_wage'], errors='coerce')

        return df
    except Exception as e:
        print(f"Error fetching occupations for {state}: {e}")
        return pd.DataFrame()

# Initialize Dash app
app = dash.Dash(__name__)

# Layout
app.layout = html.Div([
    html.H1("Top 10 Occupations in Your State", style={'color': 'orange'}),
    
    # Input field for state
    dcc.Input(id='state-input', type='text', placeholder='Enter state name', style={'width': '50%', 'margin-bottom': '20px'}),
    
    # Graph for bar chart
    dcc.Graph(id='occupation-chart')
])

# Callback for bar chart
@app.callback(
    Output('occupation-chart', 'figure'),
    [Input('state-input', 'value')]
)
def update_chart(selected_state):
    if not selected_state:
        return px.bar(title="Please enter a state name to get data. Example: Arkansas")
    
    selected_state = selected_state.strip().title()  # Normalize input
    df = get_top_occupations(selected_state)
    
    if df.empty:
        print(f"No data found for state: {selected_state}")  # Log empty data
        return px.bar(title="No data found for the selected state. Check input.")
    
    # Create a bar chart using the fetched data
    fig = px.bar(
        df,
        x='occupational_title',
        y='mean_annual_wage',
        title=f"Top 10 Occupations in {selected_state} in 2023",
        labels={'mean_annual_wage': 'Mean Annual Wage', 'occupational_title': 'Occupation'},
        text='mean_annual_wage',
        height=650)
    fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
    fig.update_layout(margin=dict(b=160))
    return fig
# Run the app
if __name__ == '__main__':
    app.run_server(host='127.0.0.1', port=10 , debug=True) #or app.run_server ?????????????????????????????????????????????????????????????????/
